<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Токенизация-и-обучение-модели-Bert" data-toc-modified-id="Токенизация-и-обучение-модели-Bert-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Токенизация и обучение модели Bert</a></span></li><li><span><a href="#Обучение-модели-Логистическая-Регрессия" data-toc-modified-id="Обучение-модели-Логистическая-Регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели Логистическая Регрессия</a></span></li><li><span><a href="#Обучение-модели-Дерево-Классификации" data-toc-modified-id="Обучение-модели-Дерево-Классификации-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучение модели Дерево Классификации</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Метрика качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
!pip install -q transformers 

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from tqdm import notebook
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [3]:
data = pd.read_csv('toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data = data.drop(['Unnamed: 0'], axis=1)

In [6]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
data.drop_duplicates()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [8]:
display(data.loc[data['toxic'] == 1].shape)
display(data.loc[data['toxic'] == 0].shape)

(16186, 2)

(143106, 2)

In [9]:
# делаем приемлемый срез датасета для комфортного обучения моделей
batch_1 = data[:2000]

## Обучение

### Токенизация и обучение модели Bert

In [10]:
# Задаём параметры для токенизатора и модели Bert с предобучением на токсик текстах 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'unitary/toxic-bert')

In [11]:
# Инициализируем  токенизатор BertTokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [12]:
# Инициализируем  модель BertModel
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Преобразуем текст в номера токенов из словаря методом encode. max_length=512 - ограничиваем максимальную длинну токенизированнго твита
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True)))

In [16]:
# Делаем токенизированные текста равной длинны
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
# Укажем нулевые и не нулевые токены. Маскируем нулевые токены
attention_mask = np.where(padded != 0, 1, 0)

In [18]:
# Задаём размер батча
batch_size = 100
# Cделаем пустой список для хранения эмбеддингов твитов
embeddings = []
# Сделаем цикл по батчам. Отображать прогресс будет функция notebook
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        # Преобразуем данные в формат тензоров
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        # Преобразуем маску в формат тензоров
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        # Передадим модели данные и маску. Для ускорения вычесления применем функцию no_grad
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        # Преобразуем элементы методом numpy() к типу numpy.array
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

In [19]:
# Функцией concatenate() соберём все эмбеддинги в матрицу признаков
features = np.concatenate(embeddings)

In [20]:
# Целевой признак
target = batch_1['toxic']

In [21]:
# Разбиение выбопрки на обучающую и тестовую
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

### Обучение модели Логистическая Регрессия

In [22]:
model_lr = LogisticRegression(solver='lbfgs', random_state=12345)

In [23]:
parameters_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                'class_weight': ['None', 'balanced']}

In [24]:
lr_grid = GridSearchCV(model_lr, parameters_lr, scoring='f1', cv=10)

In [25]:
lr_grid.fit(features_train, target_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=12345),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             scoring='f1')

In [26]:
display('F1', (lr_grid.best_score_))

'F1'

0.9258596400437007

### Обучение модели Дерево Решений

In [27]:
model_dtc = DecisionTreeClassifier(random_state=12345)

In [28]:
parameters_dtc = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy']
             }

In [29]:
dtc_grid = GridSearchCV(model_dtc, parameters_dtc, scoring='f1', cv=10)

In [30]:
dtc_grid.fit(features_train, target_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             scoring='f1')

In [31]:
display('F1', (dtc_grid.best_score_))

'F1'

0.9233675464320626

***Модель Логистическая Регрессия даёт лучшие предсказания по метрике F1, берём эту модель для тестирования***

## Выводы

In [35]:
predictions_test = lr_grid.predict(features_test)

In [36]:
f1_dtc = f1_score(target_test, predictions_test)

In [37]:
display('F1', f1_dtc)

'F1'

0.9108910891089109

***На тестовой выборке модель продемонстрировала приемлемый результат по F1 мере - выше значения 0,75***